In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets

import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


# Define MTCNN baseline
We use the default params for now

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    keep_all=True, device=device
)

# Define Dataset

In [5]:
data_dir = "data/test"

try:
    # create testing folder
    os.makedirs(data_dir)

    # create label folders
    os.makedirs(f"{data_dir}/face")
    os.makedirs(f"{data_dir}/no_face")
except:
    print("Folders already exist.")

Folders already exist.


In [22]:
from IPython.display import Image

labels = pd.read_csv("new_labels.csv")

# Get all image names

files = []
labeled_files = set(labels["filename"].values)
for (dirpath, dirnames, filenames) in os.walk("data"):
    files.extend(filenames)
    break

files = [file for file in files if file not in labeled_files]

In [23]:
import ipywidgets as widgets
from IPython.display import Image, display, clear_output

face_bttn = widgets.Button(description="Face")
no_face_bttn = widgets.Button(description="No Face")
out = widgets.Output()

count = [0]

curr_file = ''

def face_bttn_clicked(_):
    d = {'filename': files[0],
                   'label': 'face'}
    files.pop(0)
    labels.loc[len(labels)] = d

    show_widgets()
        
face_bttn.on_click(face_bttn_clicked)

def no_face_clicked(_):
    d = {'filename': files[0],
                   'label': 'no face'}
    files.pop(0)
    labels.loc[len(labels)] = d

    show_widgets()

no_face_bttn.on_click(no_face_clicked)

def show_widgets():
    clear_output(wait=True)
    buttons = widgets.HBox([face_bttn, no_face_bttn])
    
    image = widgets.Image(
        value=Image(filename=f"data/{files[0]}").data,
        format="webp",
        width=300,
        height=300
    )
    
    text = widgets.Text(f"Total labeled: {len(labels)}")
    
    display(widgets.VBox([buttons, text, image, out]))
    
    
show_widgets()

In [25]:
labels.to_csv("new_labels.csv", index=True)

In [26]:
def test(row):
    filename = row["filename"]
    label = row["label"].replace(" ", "_")
    
    os.rename(f"data/{filename}", f"{data_dir}/{label}/{filename}")

faces = labels[labels["label"] == 'face']
no_faces = labels[labels["label"] == 'no face']

try:
    faces.apply(test, axis=1)
    no_faces.apply(test, axis=1)
    
    print("Moved files to relevant folders")
except:
    print("Images are already moved")

Images are already moved


In [7]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('data/test')
idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn)

In [8]:
aligned = []
names = []

accurate = 0
for x, y in tqdm(loader):
    x_aligned = mtcnn.detect(x)
    
    if x_aligned[0] is not None and y == 0:
        accurate += 1
        
f"Accuracy is: {accurate / len(faces):8f}"


100%|██████████| 6039/6039 [14:04<00:00,  7.15it/s]


'Accuracy is: 0.928238'